<a href="https://colab.research.google.com/github/QuanPhamAnh/Coursera_Capstone/blob/main/Machine%20Learning%20with%20TensorFlow%20on%20Google%20Cloud%20Platform/Introduction%20to%20TensorFlow/TensorFlow_Dataset_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow Dataset API

**Learning Objectives**
1. Learn how to use tf.data to read data from memory
1. Learn how to use tf.data in a training loop
1. Learn how to use tf.data to read data from disk
1. Learn how to write production input pipelines with feature engineering (batching, shuffling, etc.)


In this notebook, we will start by refactoring the linear regression we implemented in the previous lab so that it takes data from a`tf.data.Dataset`, and we will learn how to implement **stochastic gradient descent** with it. In this case, the original dataset will be synthetic and read by the `tf.data` API directly  from memory.

In a second part, we will learn how to load a dataset with the `tf.data` API when the dataset resides on disk.

Each learning objective will correspond to a __#TODO__  in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/solutions/2_dataset_api.ipynb).


In [24]:
import json
import math
import os
from pprint import pprint

import numpy as np
import tensorflow as tf
print(tf.version.VERSION)

2.4.1


## Loading data from memory

### Creating the dataset

Let's consider the synthetic dataset of the previous section:

In [25]:
N_POINTS = 10
X = tf.constant(range(N_POINTS), dtype=tf.float32)
Y = 2 * X + 10

We begin with implementing a function that takes as input


- our $X$ and $Y$ vectors of synthetic data generated by the linear function $y= 2x + 10$
- the number of passes over the dataset we want to train on (`epochs`)
- the size of the batches the dataset (`batch_size`)

and returns a `tf.data.Dataset`: 

**Remark:** Note that the last batch may not contain the exact number of elements you specified because the dataset was exhausted.

If you want batches with the exact same number of elements per batch, we will have to discard the last batch by
setting:

```python
dataset = dataset.batch(batch_size, drop_remainder=True)
```

We will do that here.

**Lab Task #1:** Complete the code below to 
1. instantiate a `tf.data` dataset using [tf.data.Dataset.from_tensor_slices](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices).
2. Set up the dataset to 
  * repeat `epochs` times,
  * create a batch of size `batch_size`, ignoring extra elements when the batch does not divide the number of input elements evenly.

In [26]:
# TODO 1
def create_dataset(X, Y, epochs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y)) # TODO -- Your code here.
    dataset = dataset.repeat(epochs).batch(batch_size, drop_remainder=True) # TODO -- Your code here.
    return dataset

Let's test our function by iterating twice over our dataset in batches of 3 datapoints:

In [27]:
BATCH_SIZE = 3
EPOCH = 2

dataset = create_dataset(X, Y, epochs=1, batch_size=3)

for i, (x, y) in enumerate(dataset):
    print("x:", x.numpy(), "y:", y.numpy())
    assert len(x) == BATCH_SIZE
    assert len(y) == BATCH_SIZE
assert  EPOCH

x: [0. 1. 2.] y: [10. 12. 14.]
x: [3. 4. 5.] y: [16. 18. 20.]
x: [6. 7. 8.] y: [22. 24. 26.]


### Loss function and gradients

The loss function and the function that computes the gradients are the same as before:

In [28]:
def loss_mse(X, Y, w0, w1):
    Y_hat = w0 * X + w1
    errors = (Y_hat - Y)**2
    return tf.reduce_mean(errors)


def compute_gradients(X, Y, w0, w1):
    with tf.GradientTape() as tape:
        loss = loss_mse(X, Y, w0, w1)
    return tape.gradient(loss, [w0, w1])

### Training loop

The main difference now is that now, in the traning loop, we will iterate directly on the `tf.data.Dataset` generated by our `create_dataset` function. 

We will configure the dataset so that it iterates 250 times over our synthetic dataset in batches of 2.

**Lab Task #2:** Complete the code in the cell below to call your dataset above when training the model. Note that the `step, (X_batch, Y_batch)` iterates over the `dataset`. The inside of the `for` loop should be exactly as in the previous lab. 

In [29]:
# TODO 2
EPOCHS = 250
BATCH_SIZE = 2
LEARNING_RATE = .02

MSG = "STEP {step} - loss: {loss}, w0: {w0}, w1: {w1}\n"

w0 = tf.Variable(0.0)
w1 = tf.Variable(0.0)

dataset = create_dataset(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE) # TODO -- Your code here.

for step, (X_batch, Y_batch) in enumerate(dataset): # TODO -- Your code here.

    dw0, dw1 = compute_gradients(X_batch, Y_batch, w0, w1) # TODO -- Your code here.
    w0.assign_sub(dw0 * LEARNING_RATE)
    w1.assign_sub(dw1 * LEARNING_RATE)
    # TODO -- Your code here.

    if step % 100 == 0:
        loss = loss_mse(X, Y, w0, w1) # TODO -- Your code here.
        print(MSG.format(step=step, loss=loss, w0=w0.numpy(), w1=w1.numpy()))
        
assert loss < 0.0001
assert abs(w0 - 2) < 0.001
assert abs(w1 - 10) < 0.001

STEP 0 - loss: 331.1055908203125, w0: 0.23999999463558197, w1: 0.4399999976158142

STEP 100 - loss: 3.2297768592834473, w0: 2.55655837059021, w1: 6.674341678619385

STEP 200 - loss: 0.48055925965309143, w0: 2.2146825790405273, w1: 8.717182159423828

STEP 300 - loss: 0.07150308042764664, w0: 2.082810878753662, w1: 9.505172729492188

STEP 400 - loss: 0.010638891719281673, w0: 2.03194260597229, w1: 9.809128761291504

STEP 500 - loss: 0.001582985743880272, w0: 2.012321710586548, w1: 9.926374435424805

STEP 600 - loss: 0.00023550672631245106, w0: 2.0047526359558105, w1: 9.971602439880371

STEP 700 - loss: 3.50688278558664e-05, w0: 2.0018346309661865, w1: 9.989042282104492

STEP 800 - loss: 5.220536877459381e-06, w0: 2.000706911087036, w1: 9.995771408081055

STEP 900 - loss: 7.789132610014349e-07, w0: 2.0002737045288086, w1: 9.998367309570312

STEP 1000 - loss: 1.1540369371232373e-07, w0: 2.000105381011963, w1: 9.999371528625488

STEP 1100 - loss: 1.7078491509892046e-08, w0: 2.00004005432128

## Loading data from disk

### Use tf.data to read the CSV files

The `tf.data` API can easily read csv files using the helper function tf.data.experimental.make_csv_dataset

If you have TFRecords (which is recommended), you may use tf.data.experimental.make_batched_features_dataset

The first step is to define 

- the feature names into a list `CSV_COLUMNS`
- their default values into a list `DEFAULTS`

In [30]:
CSV_COLUMNS = [
    'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], ['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]

Let's now wrap the call to `make_csv_dataset` into its own function that will take only the file pattern (i.e. glob) where the dataset files are to be located:

**Lab Task #3:** Complete the code in the `create_dataset(...)` function below to return a `tf.data` dataset made from the `make_csv_dataset`. Have a look at the [documentation here](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset). The `pattern` will be given as an argument of the function but you should set the `batch_size`, `column_names` and `column_defaults`.

In [31]:
# TODO 3
def create_dataset(pattern):
    # TODO -- Your code here.
    dataset = tf.data.experimental.make_csv_dataset(pattern, 1, CSV_COLUMNS, DEFAULTS)
    return dataset


tempds = create_dataset('/content/taxi-train.csv')
print(tempds)

<PrefetchDataset shapes: OrderedDict([(fare_amount, (1,)), (pickup_datetime, (1,)), (pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,)), (key, (1,))]), types: OrderedDict([(fare_amount, tf.float32), (pickup_datetime, tf.string), (pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32), (key, tf.string)])>


Note that this is a prefetched dataset, where each element is an `OrderedDict` whose keys are the feature names and whose values are tensors of shape `(1,)` (i.e. vectors).

Let's iterate over the two first element of this dataset using `dataset.take(2)` and let's convert them ordinary Python dictionary with numpy array as values for more readability:

In [32]:
for data in tempds.take(2):
    pprint({k: v.numpy() for k, v in data.items()})
    print("\n")

{'dropoff_latitude': array([40.758846], dtype=float32),
 'dropoff_longitude': array([-73.961525], dtype=float32),
 'fare_amount': array([16.], dtype=float32),
 'key': array([b'502'], dtype=object),
 'passenger_count': array([2.], dtype=float32),
 'pickup_datetime': array([b'2014-02-24 18:22:00 UTC'], dtype=object),
 'pickup_latitude': array([40.746864], dtype=float32),
 'pickup_longitude': array([-73.99731], dtype=float32)}


{'dropoff_latitude': array([40.736088], dtype=float32),
 'dropoff_longitude': array([-74.00072], dtype=float32),
 'fare_amount': array([7.5], dtype=float32),
 'key': array([b'4768'], dtype=object),
 'passenger_count': array([1.], dtype=float32),
 'pickup_datetime': array([b'2013-10-05 20:55:32 UTC'], dtype=object),
 'pickup_latitude': array([40.72504], dtype=float32),
 'pickup_longitude': array([-73.99528], dtype=float32)}




### Transforming the features

What we really need is a dictionary of features + a label. So, we have to do two things to the above dictionary:

1. Remove the unwanted column "key"
1. Keep the label separate from the features

Let's first implement a function that takes as input a row (represented as an `OrderedDict` in our `tf.data.Dataset` as above) and then returns a tuple with two elements:

* The first element being the same `OrderedDict` with the label dropped
* The second element being the label itself (`fare_amount`)

Note that we will need to also remove the `key` and `pickup_datetime` column, which we won't use.

**Lab Task #4a:** Complete the code in the `features_and_labels(...)` function below. Your function should return a dictionary of features and a label. Keep in mind `row_data` is already a dictionary and you will need to remove the `pickup_datetime` and `key` from `row_data` as well.

In [33]:
UNWANTED_COLS = ['pickup_datetime', 'key']

# TODO 4a
def features_and_labels(row_data):
    label = row_data.pop(LABEL_COLUMN) # TODO -- Your code here.
    features = row_data # TODO -- Your code here.
    
    # TODO -- Your code here.
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)

    return features, label

Let's iterate over 2 examples from our `tempds` dataset and apply our `feature_and_labels`
function to each of the examples to make sure it's working:

In [34]:
for row_data in tempds.take(2):
    features, label = features_and_labels(row_data)
    pprint(features)
    print(label, "\n")
    assert UNWANTED_COLS[0] not in features.keys()
    assert UNWANTED_COLS[1] not in features.keys()
    assert label.shape == [1]

OrderedDict([('pickup_longitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-73.98348], dtype=float32)>),
             ('pickup_latitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.739594], dtype=float32)>),
             ('dropoff_longitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-73.9823], dtype=float32)>),
             ('dropoff_latitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.75331], dtype=float32)>),
             ('passenger_count',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>)])
tf.Tensor([9.3], shape=(1,), dtype=float32) 

OrderedDict([('pickup_longitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-73.986404], dtype=float32)>),
             ('pickup_latitude',
              <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.752014], dtype=float32)>),
             ('dropoff_longitude',
              <tf.

### Batching

Let's now refactor our `create_dataset` function so that it takes an additional argument `batch_size` and batch the data correspondingly. We will also use the `features_and_labels` function we implemented for our dataset to produce tuples of features and labels.

**Lab Task #4b:** Complete the code in the `create_dataset(...)` function below to return a `tf.data` dataset made from the `make_csv_dataset`. Now, the `pattern` and `batch_size` will be given as an arguments of the function but you should set the `column_names` and `column_defaults` as before. You will also apply a `.map(...)` method to create features and labels from each example. 

In [35]:
# TODO 4b
def create_dataset(pattern, batch_size):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

    dataset = dataset.map(features_and_labels) # TODO -- Your code here.

    return dataset

Let's test that our batches are of the right size:

In [36]:
BATCH_SIZE = 2

tempds = create_dataset('/content/taxi-train.csv', batch_size=2)

for X_batch, Y_batch in tempds.take(2):
    pprint({k: v.numpy() for k, v in X_batch.items()})
    print(Y_batch.numpy(), "\n")
    assert len(Y_batch) == BATCH_SIZE

{'dropoff_latitude': array([40.711983, 40.738373], dtype=float32),
 'dropoff_longitude': array([-73.99796, -73.98779], dtype=float32),
 'passenger_count': array([1., 2.], dtype=float32),
 'pickup_latitude': array([40.701515, 40.730553], dtype=float32),
 'pickup_longitude': array([-74.01231, -73.98041], dtype=float32)}
[5.3 4.5] 

{'dropoff_latitude': array([40.721462, 40.760647], dtype=float32),
 'dropoff_longitude': array([-74.01008, -73.98746], dtype=float32),
 'passenger_count': array([1., 1.], dtype=float32),
 'pickup_latitude': array([40.772 , 40.7571], dtype=float32),
 'pickup_longitude': array([-73.951744, -73.97196 ], dtype=float32)}
[20.1  8.5] 



### Shuffling

When training a deep learning model in batches over multiple workers, it is helpful if we shuffle the data. That way, different workers will be working on different parts of the input file at the same time, and so averaging gradients across workers will help. Also, during training, we will need to read the data indefinitely.

Let's refactor our `create_dataset` function so that it shuffles the data, when the dataset is used for training.

We will introduce an additional argument `mode` to our function to allow the function body to distinguish the case
when it needs to shuffle the data (`mode == 'train'`) from when it shouldn't (`mode == 'eval'`).

Also, before returning we will want to prefetch 1 data point ahead of time (`dataset.prefetch(1)`) to speed-up training:

**Lab Task #4c:** The last step of our `tf.data` dataset will specify shuffling and repeating of our dataset pipeline. Complete the code below to add these three steps to the Dataset pipeline
1. follow the `.map(...)` operation which extracts features and labels with a call to `.cache()` the result.
2. during training, use `.shuffle(...)` and `.repeat()` to shuffle batches and repeat the dataset
3. use `.prefetch(...)` to take advantage of multi-threading and speedup training.

In [37]:
# TODO 4c
def create_dataset(pattern, batch_size=1, mode='eval'):
    dataset = tf.data.experimental.make_csv_dataset(
        pattern, batch_size, CSV_COLUMNS, DEFAULTS)

    dataset = dataset.map(features_and_labels).cache() # TODO -- Your code here.

    if mode == 'train':
        dataset = dataset.shuffle(1000).repeat() # TODO -- Your code here.

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1) # TODO -- Your code here.
    
    return dataset

Let's check that our function works well in both modes:

In [38]:
tempds = create_dataset('/content/taxi-train.csv', 2, 'train')
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.97363, -73.95969], dtype=float32)>), ('pickup_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.748085, 40.77353 ], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.98135, -73.97356], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.737335, 40.782883], dtype=float32)>), ('passenger_count', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>)]), <tf.Tensor: shape=(2,), dtype=float32, numpy=array([4.5, 9.5], dtype=float32)>)]


In [39]:
tempds = create_dataset('/content/taxi-valid.csv', 2, 'eval')
print(list(tempds.take(1)))

[(OrderedDict([('pickup_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.97366, -73.98221], dtype=float32)>), ('pickup_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.762024, 40.771538], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([-73.9904 , -73.97393], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([40.740395, 40.74408 ], dtype=float32)>), ('passenger_count', <tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 4.], dtype=float32)>)]), <tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 9.7, 12.5], dtype=float32)>)]


In the next notebook, we will build the model using this input pipeline.

Copyright 2021 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.